In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [5]:
!pip install keras #ignore if already installed
#Google Colab has already inbuilt keras

In [3]:
from google.colab import drive
drive.mount('/content/drive')
# Provides link for authentication towards google drive, copy the code and paste in the box. 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
all_data=pd.read_csv("/content/drive/My Drive/Deep Learning/train.csv") #Reading csv file from G-Drive

In [5]:
all_data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(all_data['question_text'], all_data['target'], test_size=0.2, random_state=42)
#test_size decides the size of the data that has to be split as test dataset. 0.2 means 20%
#random_state is used as seed for the random number generation during split 

In [0]:
from nltk import word_tokenize

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
question_lens=[]
for question in all_data['question_text']:
    question_lens.append(len(word_tokenize(question)))

max(question_lens) 

np.quantile(question_lens,0.90)

from keras.preprocessing.text import Tokenizer

max_len = 30
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train)
sequences_train = tok.texts_to_sequences(X_train)

X_train.head()

vocab_len=len(tok.index_word.keys())

vocab_len

from keras.preprocessing import sequence

sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len)

sequences_matrix_train

sequences_matrix_train[0]

Using TensorFlow backend.


array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     9,     3,  4780, 11233,     3,   130,
          27,   427, 11233], dtype=int32)

In [0]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(vocab_len+1,300,input_length=max_len,mask_zero=True)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model
    # input layer, 2-dense layers, relu-activation function, sigmoid- activation function for output

In [0]:
from keras.layers import Input, Embedding, LSTM, Activation, Dense, Dropout
from keras.models import Model

In [31]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'])




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 30)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 30, 300)           58857900  
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                93440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
____________________________________________

In [0]:
sequences_test = tok.texts_to_sequences(X_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test,
                                               maxlen=max_len)

In [35]:
model.fit(sequences_matrix_train,y_train,batch_size=500,epochs=10,validation_data=(sequences_matrix_test,y_test))




Train on 1044897 samples, validate on 261225 samples
Epoch 1/10





1044897/1044897 [==============================] - 2277s 2ms/step - loss: 0.1264 - acc: 0.9518 - val_loss: 0.1100 - val_acc: 0.9560
Epoch 2/10
1044897/1044897 [==============================] - 2270s 2ms/step - loss: 0.0984 - acc: 0.9597 - val_loss: 0.1113 - val_acc: 0.9557
Epoch 3/10
1044897/1044897 [==============================] - 2274s 2ms/step - loss: 0.0816 - acc: 0.9665 - val_loss: 0.1200 - val_acc: 0.9550
Epoch 4/10
1044897/1044897 [==============================] - 2271s 2ms/step - loss: 0.0659 - acc: 0.9730 - val_loss: 0.1429 - val_acc: 0.9539
Epoch 5/10
1044897/1044897 [==============================] - 2278s 2ms/step - loss: 0.0522 - acc: 0.9785 - val_loss: 0.1640 - val_acc: 0.9495
Epoch 6/10
1044897/1044897 [==============================] - 2295s 2ms/step - loss: 0.0412 - acc: 0.9830 - val_loss: 0.1967 - val_acc: 0.9488
Epoch 7/10
1044897/1044897 [==============================] - 2292s 2ms/step - lo

In [0]:
predictions=model.predict(sequences_matrix_test)

In [0]:
from sklearn.metrics import roc_auc_score

In [38]:
roc_auc_score(y_test,predictions)

0.9129426764503731